In [1]:
from scipy.interpolate import make_interp_spline, BSpline
from statistics import mode
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd 

In [2]:
file_name = 'res_life.txt' #write file name with lifetime array
file = open(file_name)
string = file.readline()
data = [int(num)*0.001 for num in string.split(', ')]

In [3]:
def get_optimal_bin_number(array): #function return how much bins is optimal
    q3, q1 = np.percentile(array, [75 ,25])
    IQR = q3 - q1

    n = len(array)
    h = 2*IQR*n**(-1/3)
    n = int((max(array) - min(array))/h)
    
    return n

## Hist

In [4]:
mean = np.mean(data)
moda = mode(data)
hist, bins = np.histogram(data, bins= get_optimal_bin_number(data) )
center = bins[:-1]

xnew = np.linspace(center.min(), center.max(), 300) 
spl = make_interp_spline(center, hist, k=3) 
power_smooth = spl(xnew)

plt.figure(figsize = (14, 10), dpi = 1000)
plt.title('Distribution of the lifetime of the hydroxonium ion in water')
plt.ylabel('Number of fixed states')
plt.xlabel('Lifetime, picosec')
plt.hist(data,color='steelblue',histtype= 'stepfilled', bins= get_optimal_bin_number(data), alpha = 0.5)
plt.plot(xnew, power_smooth, color = 'b', label = 'Smoothing curve')
plt.vlines(mean, 0, 100, ls = '--', colors='k', label = 'Mean')
plt.vlines(moda, 0, 100, ls = '--', colors='r', label = 'Mode')
plt.text(0.02, 100, 'Mode = ' + str(moda) + ', Mean = ' + str(round(mean,3)), fontsize = 15)
plt.legend()
plt.savefig('Final_hist.png')

## Statistics

In [5]:
q3, q1 = np.percentile(data, [75 ,25])
IQR = q3 - q1
df = np.array(data)

statistics = pd.DataFrame()
statistics['Mean value'] = [df.mean()]
statistics['Dispersion'] = df.std()
statistics['Mode'] = moda
statistics['Median'] = np.median(df)
statistics['75%percentile'] = q3
statistics['25%percentile'] = q1
statistics['Interquartile range'] = IQR
statistics['Maximum value'] = df.max()
statistics['Minimum value'] = df.min()

In [6]:
statistics

,Mean value,Dispersion,Mode,Median,75%percentile,25%percentile,Interquartile range,Maximum value,Minimum value
0,0.115398,0.094585,0.119,0.097,0.1705,0.0385,0.132,0.427,0.001


In [7]:
import os  
statistics.to_csv('res.csv')  